In [4]:
def retrievedata(day, month, year):
    import yaml
    from influxdb import InfluxDBClient
    from influxdb import DataFrameClient
    import numpy as np
    import pandas as pd
    import csv
    from dateutil import parser
    import datetime as dt
    from datetime import timedelta

    with open("configuration.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
    
    influx = cfg['influxdb']

    client = InfluxDBClient(host=influx['host'], port=influx['port'], username=influx['username'], password=influx['password'], database=influx['database'],SSL=False)

    stre = "'" + str(year)+"-"
    if month<10:
        stre = stre + "0"
    stre = stre + str(month) + "-"
    if day<10:
        stre = stre+"0"
    stre = stre+str(day)+ "'"
    query = "Select * from EspressoPower where time >" + stre
    s = client.query(query).get_points()
    query = "Select * from GrinderMachinePower where time>" +stre
    g = client.query(query).get_points()

    date = []
    date_final = []
    date_s = []
    power_s = []
    date_g =[]
    power_g = []
    dt1 = dt.datetime(year,month,day,5,0,0) 

    while(dt1!=dt.datetime(year,month,day,18,0,0)):
        z= dt1+timedelta(hours = 2)
        z = z + timedelta(minutes = 3)
        date.append(z)
        dt1 = dt1 + timedelta(seconds=1)



    for i in s:
        d = parser.parse(i[u'time'])
        t = dt.datetime(d.year,d.month,d.day,d.hour,d.minute,d.second)
        z= t+timedelta(hours = 2)
        z = z + timedelta(minutes = 3)
        if z.day == day:
            power_s.append(i[u'value'])
            date_s.append(z)

    for i in g:
        d = parser.parse(i[u'time'])
        t = dt.datetime(d.year,d.month,d.day,d.hour,d.minute,d.second)
        z= t+timedelta(hours = 2)
        z = z + timedelta(minutes = 3)
        if z.day == day:
            power_g.append(i[u'value'])
            date_g.append(z)

    c = 0
    ps = 0
    pg = 0
    pd1 = dt.datetime(2018,5,22,0,0,0)
    power_sf=[]
    power_gf = []
    for i in date:
        if c<len(date_s):
            if pd1 == date_s[c]:
                c+=1
            if i == date_s[c]:
                pd1 = i
                ps = power_s[c]
                c+=1
            power_sf.append(ps)
        else:
            power_sf.append(ps)


    pd1 = dt.datetime(2018,5,22,0,0,0)
    c=0
    for i in date:
        if c<len(date_g):
            if pd1 == date_g[c]:
                c+=1
            if i == date_g[c]:
                pd1 = i
                pg = power_g[c]
                c+=1
            power_gf.append(pg)
        else:
            power_gf.append(pg)

    df = pd.DataFrame()
    df['time'] = date
    df['PowS']= power_sf
    df['PowG'] = power_gf
    filename = "data/espresso" + str(day) + "-" + str(month)+ "-"+str(year)+ ".csv"
    df.to_csv(filename, sep=',')
